*Analytical Information Systems*

# European Parliament Election 2019

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2019

<h1>Agenda<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Visualizing-the-European-Parliament-Election-2019" data-toc-modified-id="Visualizing-the-European-Parliament-Election-2019-1">Visualizing the European Parliament Election 2019</a></span><ul class="toc-item"><li><span><a href="#Download-the-data" data-toc-modified-id="Download-the-data-1.1">Download the data</a></span></li><li><span><a href="#Filter-and-prepare-the-data" data-toc-modified-id="Filter-and-prepare-the-data-1.2">Filter and prepare the data</a></span></li><li><span><a href="#Vote-shares---rebuilding-the-graph" data-toc-modified-id="Vote-shares---rebuilding-the-graph-1.3">Vote shares - rebuilding the graph</a></span></li><li><span><a href="#Vote-count" data-toc-modified-id="Vote-count-1.4">Vote count</a></span></li><li><span><a href="#Comparing-two-parties" data-toc-modified-id="Comparing-two-parties-1.5">Comparing two parties</a></span></li><li><span><a href="#Relationship" data-toc-modified-id="Relationship-1.6">Relationship</a></span></li></ul></li></ul></div>

## Visualizing the European Parliament Election 2019

The ninth elections to the European Parliament in Germany was held on 26 May 2019, electing members of the national Germany constituency to the European Parliament. The results are available [here](https://www.bundeswahlleiter.de/europawahlen/2019/ergebnisse.html).

<img src="images/03/Stimmenanteile.png" style="width:100%">

### Download the data

In [ ]:
library(tidyverse)
'https://www.bundeswahlleiter.de/dam/jcr/5441f564-1f29-4971-9ae2-b8860c1724d1/ew19_kerg2.csv' %>% 
    read_csv2(skip = 9) -> eu_elections

Get an impression of the dataset

In [ ]:
glimpse(eu_elections)
eu_elections$Gebietsart %>% unique()
eu_elections$Gruppenart %>% unique()

### Filter and prepare the data

In [ ]:
eu_elections %>%
    filter(Gebietsart == 'Bund', Gruppenart == 'Partei') %>%
    mutate(Gruppenname = fct_reorder(Gruppenname, -VorpProzent), 
           Gruppenname = fct_lump(Gruppenname, prop=0.0065, w = Prozent, other_level = "Sonstige")) %>%
    group_by(Gruppenname) %>%
    summarise(Prozent = sum(Prozent),
              VorpProzent =  sum(VorpProzent),
              Anzahl =  sum(Anzahl),
              VorpAnzahl =  sum(Anzahl)) -> share_data
share_data 

How would you solve this using the `if_else()` function? You may need `fct_relevel()` to reorder the factor levels.

```R
    mutate(Gruppenname = if_else(Prozent < 0.65, 'Sonstige', Gruppenname),
           Gruppenname = fct_reorder(Gruppenname, -VorpProzent),
           Gruppenname = fct_relevel(Gruppenname, "Sonstige" , after = Inf))
```

Colors have a special meaning in our democracy. There is no universal standard to map the parties to their respective colors, so we have to do it manually (here, in a named vector)

In [ ]:
party_colors <- c(
    'AfD'= 'turquoise',
    'CDU'= 'darkblue',
    'CSU'='blue',
    'DIE LINKE'='purple',
    'Die PARTEI'= 'darkred',
    'FAMILIE'='pink',
    'FDP'='yellow',
    'FREIE WÄHLER'='lightblue',
    'GRÜNE'='seagreen',
    'PIRATEN'= 'orange',
    'SPD'='red',
    'Tierschutzpartei'='lightgreen',
    'Volt'='blue',
    'ÖDP'= 'orange2',
    'Sonstige'= 'gray')

### Vote shares - rebuilding the graph

In [ ]:
options(repr.plot.width=8, repr.plot.height=3)
share_data %>% 
    ggplot(aes(x = fct_reorder(Gruppenname, -VorpProzent), y = Prozent, fill=Gruppenname)) + 
    geom_col(width=0.5) + 
    geom_col(aes(y = VorpProzent), alpha=0.5, position= position_nudge(x = 0.25), width=0.5) +
    geom_text(aes(label=format(round(Prozent, 1), nsmall = 1)), size= 3, vjust = -0.5) +
    scale_fill_manual(values= party_colors) + 
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 0.95)) +
    xlab(label = element_blank()) +
    guides(fill=FALSE)

### Vote count

In [ ]:
share_data %>%
    mutate(Gruppenname = fct_reorder(Gruppenname, Anzahl),
           Gruppenname = fct_relevel(Gruppenname, "Sonstige")) %>%
    ggplot(aes(x=Gruppenname, y=Anzahl/1000000, fill=Gruppenname)) + 
    geom_col(width=0.7) + 
    geom_text(aes(label=format(round(Anzahl/1000000, 2), nsmall = 2)), size= 3, hjust = -0.1) +
    scale_fill_manual(values= party_colors) + 
    theme_minimal() +
    xlab(label = element_blank()) +
    ylab(label = 'Stimmen in Mio.') +
    guides(fill=FALSE) +
    coord_flip()

### Comparing two parties   

What is the distribution of shares in the different constituencies of each federal state?

First, we need to get map the federal state codes to their respective names:

In [ ]:
eu_elections %>%
    filter(Gebietsart == 'Land') %>%
    select(Gebietsnummer, Gebietsname) %>%
    rename(Land = Gebietsname) %>%
    distinct() -> lookUpLand
lookUpLand

__Plot the comparison:__ distribution of shares in the different constituencies of each federal state of the parties "AfD" and "GRÜNE".

In [ ]:
eu_elections %>%
    filter(Gebietsart == 'Kreis', Gruppenart == 'Partei') %>%
    filter(Gruppenname %in% c("AfD", "GRÜNE")) %>%
    left_join(lookUpLand, by=c("UegGebietsnummer" = "Gebietsnummer")) %>%
    ggplot(aes(x=Prozent, fill=Gruppenname)) + 
    geom_density(alpha=0.5) + 
    scale_fill_manual(values= party_colors) + 
    theme_minimal() +
    facet_wrap(~Land)

### Relationship

Is there any relationship beweeen the shares of two parties at constituency level?

Filter and preprocess the data

In [ ]:
eu_elections %>%
    filter(Gebietsart == 'Kreis', Gruppenart == 'Partei') %>%
    select(Gebietsname, Gruppenname, Prozent) %>%
    spread(Gruppenname, Prozent) -> kreis_data
kreis_data %>% head()

Plot the relationship beweeen the shares of 'GRÜNE' and 'CSU' at constituency level.

In [ ]:
set.seed(0)
kreis_data %>% 
    ggplot(aes(x=CSU,y=GRÜNE, label=Gebietsname)) + 
    geom_point() + 
    ggrepel::geom_text_repel(data=sample_frac(kreis_data, 0.15),alpha=0.8) +
    theme_minimal()